In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
   .appName("Neural Network Model") \
   .config("spark.executor.memory", "3gb") \
   .getOrCreate()
   
sc = spark.sparkContext

In [ ]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

**2. Data Understanding using SparkSQL**

In [ ]:
! wget https://storage.googleapis.com/class25jan2022/share/2008.csv

--2022-02-12 02:55:11--  https://storage.googleapis.com/class25jan2022/share/2008.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.125.128, 142.250.157.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689413344 (657M) [text/csv]
Saving to: ‘2008.csv’

2008.csv            100%[===================>] 657.48M   115MB/s    in 9.2s    

2022-02-12 02:55:22 (71.3 MB/s) - ‘2008.csv’ saved [689413344/689413344]



In [ ]:
! wc -l ./2008.csv

7009729 ./2008.csv


In [ ]:
! head -3 2008.csv

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2008,1,3,4,2003,1955,2211,2225,WN,335,N712SW,128,150,116,-14,8,IAD,TPA,810,4,8,0,,0,NA,NA,NA,NA,NA
2008,1,3,4,754,735,1002,1000,WN,3231,N772SW,128,145,113,2,19,IAD,TPA,810,5,10,0,,0,NA,NA,NA,NA,NA


In [ ]:
raw_df = spark.read.format('csv').\
option('header','true').option('mode','DROPMALFORMED')\
.load('2008.csv')

In [ ]:
raw_df.count()

7009728

In [ ]:
raw_df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

**3. Data Preparation using SparkSQL**

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
crunched_df = raw_df.\
withColumn('DepTime',col('DepTime').\
           cast(DoubleType())).\
withColumn('TaxiOut',col('TaxiOut').\
           cast(DoubleType())).\
withColumn('TaxiIn',col('TaxiIn').\
           cast(DoubleType())).\
withColumn('DepDelay',col('DepDelay').\
           cast(DoubleType())).\
withColumn('DayOfWeek',col('DayOfWeek').\
           cast(DoubleType())).\
withColumn('Distance',col('Distance').\
           cast(DoubleType())).\
withColumn('ArrDelay',col('ArrDelay').\
           cast(DoubleType()))

In [ ]:
crunched_df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: double (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- TaxiIn: double (nullable = true)
 |-- TaxiOut: double (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

In [ ]:
crunched_df\
.select(['DepTime','TaxiOut','TaxiIn',\
                     'DayOfWeek','Distance','ArrDelay'])\
                     .describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
DepTime,6873482,1333.8300461105448,478.06889486629836,1.0,2400.0
TaxiOut,6872670,16.453045177492882,11.332798654232155,0.0,429.0
TaxiIn,6858079,6.860851704974527,4.933649371300466,0.0,308.0
DayOfWeek,7009728,3.9241815088973495,1.9882589459851212,1.0,7.0
Distance,7009728,726.3870294253928,562.1018034840403,11.0,4962.0
ArrDelay,6855029,8.16845238729114,38.50193694882867,-519.0,2461.0


In [ ]:
def t_timeperiod(origin):
    if origin is None:
        period = None
    elif origin > 0 and origin < 600:
        period = '00.01-05.59'
    elif origin >= 600 and origin < 1200:
        period = '06.00-11.59'
    elif origin >= 1200 and origin < 1800:
        period = '12.00-17.59'
    elif origin >= 1800 and origin <= 2400:
        period = '18.00-24.00'
    else:
        period = 'NA'
    return period

In [ ]:
timeperiod = udf(lambda x: t_timeperiod(x),StringType())

In [ ]:
discretized_df = crunched_df.\
withColumn('DepTime',timeperiod(crunched_df['DepTime']))

In [69]:
from pyspark.sql.functions import *
max_distance = discretized_df.select(max('Distance')).collect()[0][0]
min_distance = discretized_df.select(min('Distance')).collect()[0][0]

KeyboardInterrupt: ignored

In [ ]:
max_ArrDelay = discretized_df.select(max('ArrDelay')).collect()[0][0]
min_ArrDelay = discretized_df.select(min('ArrDelay')).collect()[0][0]

In [ ]:
def t_normalized_distance(origin):
    if origin is None:
        return None
    else:
        return ((origin-min_distance)/(max_distance-min_distance))

In [ ]:
def t_normalized_ArrDelay(origin):
    if origin is None:
        return None
    else:
        return ((origin-min_ArrDelay)/(max_ArrDelay-min_ArrDelay))

In [ ]:
normalized_distance = udf(lambda x: t_normalized_distance(x),DoubleType())

In [ ]:
normalized_ArrDelay = udf(lambda x: t_normalized_ArrDelay(x),DoubleType())

In [ ]:
normalized_df = discretized_df.\
withColumn('Distance', normalized_distance(discretized_df['Distance'])).\
withColumn('ArrDelay', normalized_ArrDelay(discretized_df['ArrDelay']))

In [ ]:
features_df = normalized_df.\
select(['UniqueCarrier','Origin','Dest',\
        'DepTime','TaxiOut','TaxiIn','DepDelay',\
        'DayOfWeek','Distance','ArrDelay'])

In [ ]:
final_df = features_df.dropna()

In [ ]:
final_df.count()

6855029

In [ ]:
final_df.show()

+-------------+------+----+-----------+-------+------+--------+---------+--------------------+-------------------+
|UniqueCarrier|Origin|Dest|    DepTime|TaxiOut|TaxiIn|DepDelay|DayOfWeek|            Distance|           ArrDelay|
+-------------+------+----+-----------+-------+------+--------+---------+--------------------+-------------------+
|           WN|   IAD| TPA|18.00-24.00|    8.0|   4.0|     8.0|      4.0| 0.16138153908301353|0.16946308724832215|
|           WN|   IAD| TPA|06.00-11.59|   10.0|   5.0|    19.0|      4.0| 0.16138153908301353|0.17483221476510066|
|           WN|   IND| BWI|06.00-11.59|   17.0|   3.0|     8.0|      4.0|  0.1017976166431024|0.17885906040268457|
|           WN|   IND| BWI|06.00-11.59|    7.0|   3.0|    -4.0|      4.0|  0.1017976166431024|0.17214765100671142|
|           WN|   IND| BWI|18.00-24.00|   10.0|   3.0|    34.0|      4.0|  0.1017976166431024| 0.1855704697986577|
|           WN|   IND| JAX|18.00-24.00|   10.0|   4.0|    25.0|      4.0|  0.136

4. Modeling (Spark ML and Spark ML Pipeline)

In [ ]:
training_df,test_df = final_df.\
randomSplit([0.80,0.20], seed = 12)

In [ ]:
training_df.count()

5484163

In [ ]:
training_df.printSchema()

root
 |-- UniqueCarrier: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- TaxiOut: double (nullable = true)
 |-- TaxiIn: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- DayOfWeek: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- ArrDelay: double (nullable = true)



In [ ]:
from pyspark.ml.feature import StringIndexer,OneHotEncoder

In [ ]:
DepTimeIndexer = StringIndexer\
(inputCol='DepTime',outputCol='DepTimeIndexed')

In [ ]:
DepTimeOneHotEncoder = OneHotEncoder\
(dropLast=False,inputCol='DepTimeIndexed', \
 outputCol='DepTimeVec')

In [ ]:
UniqueCarrierIndexer = StringIndexer\
(inputCol='UniqueCarrier',\
 outputCol='UniqueCarrierIndexed')

In [ ]:
UniqueCarrierOneHotEncoder = OneHotEncoder\
(dropLast=False,inputCol='UniqueCarrierIndexed',\
 outputCol='UniqueCarrierVec')

In [ ]:
OriginIndexer = StringIndexer(inputCol='Origin',\
                              outputCol='OriginIndexed')

In [ ]:
OriginOneHotEncoder = OneHotEncoder\
(dropLast=False,inputCol='OriginIndexed',\
 outputCol='OriginVec')

In [ ]:
DestIndexer = StringIndexer(inputCol='Dest',\
                            outputCol='DestIndexed')

In [ ]:
DestOneHotEncoder = OneHotEncoder\
(dropLast=False,inputCol='DestIndexed',\
 outputCol='DestVec')

In [ ]:
from pyspark.mllib.linalg import Vectors

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
featureAssembler = VectorAssembler\
(inputCols=['UniqueCarrierVec',\
            'OriginVec',\
            'DepTimeIndexed',\
            'DestVec',\
            'TaxiOut','TaxiIn',\
            'DepDelay',\
            'DayOfWeek',\
            'Distance'
           ], outputCol='***features')

In [ ]:
from pyspark.ml.regression import RandomForestRegressor

In [ ]:
dt = RandomForestRegressor\
(labelCol='ArrDelay',featuresCol='***features')

In [ ]:
pipeline_dt = Pipeline().\
setStages([UniqueCarrierIndexer,\
           UniqueCarrierOneHotEncoder,\
           DepTimeIndexer,\
           OriginIndexer ,\
           OriginOneHotEncoder,\
           DestIndexer,\
           DestOneHotEncoder,\
           featureAssembler,dt])

In [ ]:
dtModel = pipeline_dt.fit(training_df)

5. Model Evalution

In [ ]:
result_df = dtModel.transform(test_df.dropna())

In [ ]:
result_df.select(['ArrDelay','Prediction']).show(10)

+-------------------+-------------------+
|           ArrDelay|         Prediction|
+-------------------+-------------------+
|0.17348993288590603|0.17287572364452058|
|0.18053691275167785| 0.1758793947109317|
|0.17013422818791946| 0.1730034432629028|
| 0.1714765100671141|0.17346233179077675|
|0.17651006711409395|0.17437803271824098|
|0.17583892617449665|0.17615346340264085|
|0.18523489932885906|0.18379687727395563|
|  0.174496644295302|0.17413089607186197|
|0.17315436241610738|0.17523882388087936|
|0.16946308724832215|0.17395417166518642|
+-------------------+-------------------+
only showing top 10 rows



In [56]:
from pyspark.ml.evaluation import RegressionEvaluator

In [57]:
everesult_df = dtModel.transform(test_df.dropna())

In [58]:
lr_evaluator_r2 = RegressionEvaluator\
(predictionCol="prediction",labelCol="ArrDelay",metricName="r2")
print("R Squared (R2) on test data = %g" \
      % lr_evaluator_r2.evaluate(everesult_df))

R Squared (R2) on test data = 0.71507


In [59]:
lr_evaluator_rmse = RegressionEvaluator\
(predictionCol="prediction",labelCol="ArrDelay",metricName="rmse")
print("Root Mean Squared Error (RMSE) on test data = %g" \
      % lr_evaluator_rmse.evaluate(everesult_df))

Root Mean Squared Error (RMSE) on test data = 0.0069116


##HOMEWORK 4: Improving model performance


In [64]:
#1st normalize all numerical features (TaxiOut|TaxiIn|DepDelay|DayOfWeek)
#2nd algorithm = linear regression

In [70]:
max_taxiout = final_df.select(max('TaxiOut')).collect()[0][0]
min_taxiout = final_df.select(min('TaxiOut')).collect()[0][0]

max_taxiin = final_df.select(max('TaxiIn')).collect()[0][0]
min_taxiin = final_df.select(min('TaxiIn')).collect()[0][0]

max_DepDelay = final_df.select(max('DepDelay')).collect()[0][0]
min_DepDelay = final_df.select(min('DepDelay')).collect()[0][0]

max_DayOfWeek = final_df.select(max('DayOfWeek')).collect()[0][0]
min_DayOfWeek = final_df.select(min('DayOfWeek')).collect()[0][0]

In [79]:
def t_normalized_taxiout(origin):
    if origin is None:
        return None
    else:
        return ((origin-min_taxiout)/(max_taxiout-min_taxiout))

def t_normalized_taxiin(origin):
    if origin is None:
        return None
    else:
        return ((origin-min_taxiin)/(max_taxiin-min_taxiin))

def t_normalized_DepDelay(origin):
    if origin is None:
        return None
    else:
        return ((origin-min_DepDelay)/(max_DepDelay-min_DepDelay))

def t_normalized_DayOfWeek(origin):
    if origin is None:
        return None
    else:
        return ((origin-min_DayOfWeek)/(max_DayOfWeek-min_DayOfWeek))

In [83]:
normalized_taxiout = udf(lambda x: t_normalized_taxiout(x),DoubleType())

normalized_taxiin = udf(lambda x: t_normalized_taxiin(x),DoubleType())

normalized_DepDelay = udf(lambda x: t_normalized_DepDelay(x),DoubleType())

normalized_DayOfWeek = udf(lambda x: t_normalized_DayOfWeek(x),DoubleType())

In [84]:
final_df2 = final_df.\
withColumn('TaxiOut', normalized_taxiout(final_df['TaxiOut'])).\
withColumn('TaxiIn', normalized_taxiin(final_df['TaxiIn'])).\
withColumn('DepDelay', normalized_DepDelay(final_df['DepDelay'])).\
withColumn('DayOfWeek', normalized_DayOfWeek(final_df['DayOfWeek']))

In [87]:
final_df2.show()

+-------+-------------+-------+-------+-----------+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+
|summary|UniqueCarrier| Origin|   Dest|    DepTime|             TaxiOut|             TaxiIn|            DepDelay|          DayOfWeek|           Distance|            ArrDelay|
+-------+-------------+-------+-------+-----------+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+
|  count|      6855029|6855029|6855029|    6855029|             6855029|            6855029|             6855029|            6855029|            6855029|             6855029|
|   mean|         null|   null|   null|       null| 0.03833026652074983|0.02227321514075802|    0.18124753247614|0.48763365309360956| 0.1449694675338908| 0.17690216523079222|
| stddev|         null|   null|   null|       null|0.026365082778643384|0.01601046650097971|0.011730710860230004|0.3314099011

##New Model

In [88]:
training_df2,test_df2 = final_df2.\
randomSplit([0.80,0.20], seed = 12)

In [89]:
#Linear regression
from pyspark.ml.regression import LinearRegression
lr = LinearRegression\
(labelCol='ArrDelay',featuresCol='***features')

In [90]:
pipeline_lr = Pipeline().\
setStages([UniqueCarrierIndexer,\
           UniqueCarrierOneHotEncoder,\
           DepTimeIndexer,\
           OriginIndexer ,\
           OriginOneHotEncoder,\
           DestIndexer,\
           DestOneHotEncoder,\
           featureAssembler,lr])

In [91]:
lrModel = pipeline_lr.fit(training_df2)

In [92]:
result_df2 = lrModel.transform(test_df2.dropna())

In [93]:
result_df2.select(['ArrDelay','Prediction']).show(10)

+-------------------+-------------------+
|           ArrDelay|         Prediction|
+-------------------+-------------------+
|0.17348993288590603|0.17154205214602974|
|0.18053691275167785| 0.1790834208865687|
|0.17013422818791946|  0.172762565481731|
| 0.1714765100671141| 0.1729673289054487|
|0.17651006711409395|0.17679872891270176|
|0.17583892617449665|0.17673271389452255|
|0.18523489932885906|0.18750823274834252|
|  0.174496644295302|0.17395881051922896|
|0.17315436241610738| 0.1732770397707348|
|0.16946308724832215| 0.1689061111401872|
+-------------------+-------------------+
only showing top 10 rows



In [94]:
everesult_df2 = lrModel.transform(test_df2.dropna())

In [95]:
lr_evaluator_r2 = RegressionEvaluator\
(predictionCol="prediction",labelCol="ArrDelay",metricName="r2")
print("R Squared (R2) on test data = %g" \
      % lr_evaluator_r2.evaluate(everesult_df2))

R Squared (R2) on test data = 0.944345


In [96]:
lr_evaluator_rmse = RegressionEvaluator\
(predictionCol="prediction",labelCol="ArrDelay",metricName="rmse")
print("Root Mean Squared Error (RMSE) on test data = %g" \
      % lr_evaluator_rmse.evaluate(everesult_df2))

Root Mean Squared Error (RMSE) on test data = 0.00305466


##Baseline performance

##R Squared (R2) on test data = 0.71507
##Root Mean Squared Error (RMSE) on test data = 0.0069116

##After normalization of all numerical featured and fitting with LinearRegression

##R Squared (R2) on test data = 0.944345
##Root Mean Squared Error (RMSE) on test data = 0.00305466

##Next Move

#use standardscaler
#use poylnomial features
#feature engineering
#try another algorithm eg gradient boost

In [ ]:
from pyspark.ml.feature import StandardScaler
'''discretized_df
then.....select(['UniqueCarrier','Origin','Dest',\
        'DepTime','TaxiOut','TaxiIn','DepDelay',\
        'DayOfWeek','Distance','ArrDelay'])
then...dropna        
'''

final_df3 = discretized_df.select(['UniqueCarrier','Origin','Dest',\
        'DepTime','TaxiOut','TaxiIn','DepDelay',\
        'DayOfWeek','Distance','ArrDelay']).\
        dropna()

cols_to_scale = ['TaxiOut','TaxiIn','DepDelay','DayOfWeek','Distance','ArrDelay']   #TaxiOut|TaxiIn|DepDelay|DayOfWeek|            Distance|           ArrDelay|
cols_to_keep_unscaled = ['UniqueCarrier','Origin','Dest','DepTime']     #UniqueCarrier|Origin|Dest|    DepTim

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
assembler = VectorAssembler().setInputCols(cols_to_scale).setOutputCol("features")
final_df3_transformed = assembler.transform(final_df3)
scaler_model = scaler.fit(final_df3_transformed.select("features"))
final_df3_scaled = scaler_model.transform(final_df3_transformed)

def extract(row):
  return (row.a, row.b,) + tuple(row.scaledFeatures.toArray().tolist())

final_df3_scaled = final_df3_scaled.select(*cols_to_keep_unscaled, "scaledFeatures").rdd \
        .map(extract).toDF(cols_to_keep_unscaled + cols_to_scale)
  
  
final_df3_scaled.describe().show()
 

In [ ]:
training_df3,test_df3 = final_df3.\
randomSplit([0.80,0.20], seed = 12)

In [ ]:
lrModel = pipeline_lr.fit(training_df3)

In [ ]:
result_df3 = lrModel.transform(test_df3.dropna())

In [ ]:
result_df3.select(['ArrDelay','Prediction']).show(10)

In [ ]:
everesult_df3 = lrModel.transform(test_df3.dropna())

In [ ]:
lr_evaluator_r2 = RegressionEvaluator\
(predictionCol="prediction",labelCol="ArrDelay",metricName="r2")
print("R Squared (R2) on test data = %g" \
      % lr_evaluator_r2.evaluate(everesult_df3))

In [ ]:
lr_evaluator_rmse = RegressionEvaluator\
(predictionCol="prediction",labelCol="ArrDelay",metricName="rmse")
print("Root Mean Squared Error (RMSE) on test data = %g" \
      % lr_evaluator_rmse.evaluate(everesult_df3))